In [1]:
import optuna
import numpy as np
import time

# Schwefel function
def schwefel(trial):
    # Определяем количество признаков
    dim = 10
    # Генерируем значения признаков
    x = [trial.suggest_float(f"x{i}", -500, 500) for i in range(dim)]
    
    # Вычисляем значение функции Швефеля
    return 418.9829 * dim - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

# Словарь для хранения результатов всех итераций
results = []

# Запуск оптимизации
def optimize_schwefel():
    start_time = time.time()  # Засекаем время начала
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    
    # Внутренняя функция для сохранения результата на каждой итерации
    def callback(study, trial):
        # Сохраняем результаты каждой итерации
        results.append({
            'trial_number': trial.number,
            'value': trial.value,
            'params': trial.params,
            'time': time.time() - start_time  # Засекаем время выполнения каждой итерации
        })

    study.optimize(schwefel, n_trials=1000, callbacks=[callback])  # 1000 итераций

    # Находим оптимальный результат
    best_trial = study.best_trial
    best_result = None
    for result in results:
        if result['trial_number'] == best_trial.number:
            best_result = result
            break

    # Итоговый вывод
    print(f"Оптимальная итерация: {best_result['trial_number']}")
    print(f"Время до оптимальной итерации: {best_result['time']:.2f} секунд")
    print(f"Значение оптимальной функции: {best_result['value']}")
    print(f"Оптимальные значения признаков: {best_result['params']}")
    print(f"Всего итераций: {len(results)}")

# Запуск оптимизации
optimize_schwefel()


[I 2024-09-10 21:35:07,709] A new study created in memory with name: no-name-862bd877-69f1-466e-9de3-5ea3110637d1
[I 2024-09-10 21:35:07,712] Trial 0 finished with value: 3613.8390165786814 and parameters: {'x0': -125.45988115263754, 'x1': 450.71430640991616, 'x2': 231.99394181140508, 'x3': 98.65848419703661, 'x4': -343.98135955756345, 'x5': -344.00547966379736, 'x6': -441.91638783180053, 'x7': 366.1761457749352, 'x8': 101.11501174320881, 'x9': 208.0725777960455}. Best is trial 0 with value: 3613.8390165786814.
[I 2024-09-10 21:35:07,717] Trial 1 finished with value: 3943.234680251851 and parameters: {'x0': -479.4155057041975, 'x1': 469.9098521619943, 'x2': 332.4426408004217, 'x3': -287.66088932172386, 'x4': -318.1750327928994, 'x5': -316.5954901465662, 'x6': -195.75775704046225, 'x7': 24.756431632237877, 'x8': -68.05498135788423, 'x9': -208.77085980195807}. Best is trial 0 with value: 3613.8390165786814.
[I 2024-09-10 21:35:07,719] Trial 2 finished with value: 4722.709461616976 and pa

Оптимальная итерация: 677
Время до оптимальной итерации: 68.69 секунд
Значение оптимальной функции: 683.4084926289565
Оптимальные значения признаков: {'x0': 186.6134095391541, 'x1': 446.06961021704575, 'x2': 411.2550451402264, 'x3': 426.31816988205776, 'x4': 205.87382930766174, 'x5': 401.6143511654963, 'x6': 437.57162451915104, 'x7': 417.5477297412015, 'x8': 403.93780805025415, 'x9': 424.40079783059105}
Всего итераций: 1000
